In [2]:
# !conda activate tracer
# !export IGDATA=/home/laura/projects/ncbi-igblast-1.21.0/bin

Try PB library B -> donor 388 day 14  
Try PB library P -> donor 388 day 28

### Identify barcodes of interest

In [3]:
import anndata

In [4]:
vdj_adata = anndata.read(f"/home/laura/projects/COH_data/alignment_v2/VDJ/all_norm.h5ad")
vdj_adata

AnnData object with n_obs × n_vars = 113477 × 17549
    obs: 'species', 'tissue', 'library_id', 'sample', 'n_genes', 'cell_barcode', 'donor', 'batch_index', 'timepoint'
    var: 'gene_name', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
    layers: 'spliced', 'unspliced'

Only keep filtered barcodes from donor 388, day 14 library B:

In [19]:
vdj_adata.obs[(vdj_adata.obs["donor"] == "388") & (vdj_adata.obs["timepoint"] == "14") & (vdj_adata.obs["sample"] == "B")]

,species,tissue,library_id,sample,n_genes,cell_barcode,donor,batch_index,timepoint
barcode,,,,,,,,,
AAACCTGGTCAATACC-1,homo_sapiens,PB,SE6453_SA71968,B,498,AAACCTGGTCAATACC,388,1,14
AAAGCAAAGCCACGCT-1,homo_sapiens,PB,SE6453_SA71968,B,17,AAAGCAAAGCCACGCT,388,1,14
AAAGCAATCATCTGTT-1,homo_sapiens,PB,SE6453_SA71968,B,33,AAAGCAATCATCTGTT,388,1,14
AAAGCAATCGCAGGCT-1,homo_sapiens,PB,SE6453_SA71968,B,19,AAAGCAATCGCAGGCT,388,1,14
AAAGTAGGTGATGTCT-1,homo_sapiens,PB,SE6453_SA71968,B,19,AAAGTAGGTGATGTCT,388,1,14
...,...,...,...,...,...,...,...,...,...
TTTGCGCTCTGGCGAC-1,homo_sapiens,PB,SE6453_SA71968,B,32,TTTGCGCTCTGGCGAC,388,1,14
TTTGGTTAGAGGGCTT-1,homo_sapiens,PB,SE6453_SA71968,B,34,TTTGGTTAGAGGGCTT,388,1,14
TTTGGTTTCCACGTGG-1,homo_sapiens,PB,SE6453_SA71968,B,21,TTTGGTTTCCACGTGG,388,1,14


In [14]:
barcodes_to_keep = [barcode.split("-")[0] for barcode in vdj_adata.obs[(vdj_adata.obs["donor"] == "388") & (vdj_adata.obs["timepoint"] == "14") & (vdj_adata.obs["sample"] == "B")].index.values]

In [24]:
keep_txt = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer/PB_B_388_d14_keep.txt"

In [25]:
with open(keep_txt, "w") as keep_file:
    for bc in barcodes_to_keep:
        keep_file.write(bc + "\t" + bc + "\n")

In [20]:
config_txt = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer/PB_B_388_d14_config.txt"

In [21]:
with open(config_txt, "w") as config_file:
    config_file.write("tags\tlocations\n")
    for bc in barcodes_to_keep:
        config_file.write(bc + "\t0:0:16\n")

### Separate each cell of interest into a separate fastq

In [26]:
# Original fastq files
b_fastq1 = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ/FT-SE6453/FT-SA71968-FT-SPN01104_HC75HCCX2/SE6453_SA71968_S2_L008_R1_001.fastq.gz"
b_fastq2 = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ/FT-SE6453/FT-SA71968-FT-SPN01104_HC75HCCX2/SE6453_SA71968_S2_L008_R2_001.fastq.gz"

# p_fastq1 = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ/SE8002/FT-SA157958-FT-SPN03558_HHYFFCCX2/SE8002_SA157958_S4_L008_R1_001.fastq.gz"
# p_fastq2 = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ/SE8002/FT-SA157958-FT-SPN03558_HHYFFCCX2/SE8002_SA157958_S4_L008_R2_001.fastq.gz"

In [30]:
splitcode = "/home/laura/projects/splitcode/build/src/splitcode"
out_folder = "/home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer"

In [32]:
!cd $out_folder \
    && $splitcode \
        -t 30 \
        -c $config_txt \
        -k $keep_txt \
        -N 2 \
        --gzip \
        --no-output \
        $b_fastq1 \
        $b_fastq2

* Using a list of 660 tags (vector size: 660; map size: 660; num elements in map: 660)
* will process sample 1: /home/laura/projects/COH_data/alignment_v2/raw/VDJ/FT-SE6453/FT-SA71968-FT-SPN01104_HC75HCCX2/SE6453_SA71968_S2_L008_R1_001.fastq.gz
                         /home/laura/projects/COH_data/alignment_v2/raw/VDJ/FT-SE6453/FT-SA71968-FT-SPN01104_HC75HCCX2/SE6453_SA71968_S2_L008_R2_001.fastq.gz
30M reads processed         
done 
* processed 30,992,796 reads


### Run tracer assemble

Questions:  
- Tracer by default checks for alpha and beta loci but not gamma and delta loci -> should I check for those as well?

In [ ]:
tracer assemble \
    -p 30 \
    -c /home/laura/projects/tracer/tracer.conf \
    -s Hsap \
    -q kallisto \
    /home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer/AAACCTGGTCAATACC_0.fastq.gz \
    /home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer/AAACCTGGTCAATACC_1.fastq.gz \
    AAACCTGGTCAATACC \
    /home/laura/projects/COH_data/alignment_v2/raw/VDJ_tracer_results